## LIBRARIES IMPORT

In [1]:
import pandas as pd
import numpy as np
import os 
import re

## READING FILES

In [2]:
train_review =[]
for line in open('../movie-sentiment-analysis/aclImdb/movie_data/full_train.txt','r',encoding='utf-8'):
    train_review.append(line.strip())

In [3]:
test_review=[]
for line in open('../movie-sentiment-analysis/aclImdb/movie_data/full_test.txt','r',encoding='utf-8'):
    test_review.append(line.strip())

### CLEANING && PREPROCESS 

In [4]:
no_space = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])|(\d+)")
with_space = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")
def cleaningandprocessing(reviews):
    reviews = [no_space.sub("",line.lower()) for line in reviews]
    reviews = [with_space.sub(" ",line) for line in reviews]
    return reviews
clean_train_reviews = cleaningandprocessing(train_review)
clean_test_reviews = cleaningandprocessing(test_review)

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.linear_model import LogisticRegression
import nltk

In [10]:
def get_lemmatized_text(corpus):
    nltk.download('wordnet')
    from nltk.stem import WordNetLemmatizer
    lemmatizer = WordNetLemmatizer()
    return [' '.join([lemmatizer.lemmatize(word) for word in review.split()]) for review in corpus]

lemmatized_reviews_train = get_lemmatized_text(clean_train_reviews)
lemmatized_reviews_test = get_lemmatized_text(clean_test_reviews)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\srija\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\srija\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [11]:
cv = CountVectorizer(binary=True)
cv.fit(lemmatized_reviews_train)
X_train = cv.transform(lemmatized_reviews_train)
X_test = cv.transform(lemmatized_reviews_test)
target = [1 if i <12500 else 0 for i in range(25000)]
x_train, x_test, y_train, y_test = train_test_split(X_train, target, test_size=0.8, random_state=42)

In [13]:
for c in [0.01,0.05,0.1,0.25,0.5,1]:
    lr = LogisticRegression(C=c)
    lr.fit(x_train, y_train)
    print("For: ", c)
    print("\n")
    print(accuracy_score(y_test, lr.predict(x_test)))
    print("\n")
    print(confusion_matrix(y_test, lr.predict(x_test)))
    print("\n")
    print(classification_report(y_test, lr.predict(x_test)))
    print("\n\n")

For:  0.01


0.8463


[[8339 1623]
 [1451 8587]]


             precision    recall  f1-score   support

          0       0.85      0.84      0.84      9962
          1       0.84      0.86      0.85     10038

avg / total       0.85      0.85      0.85     20000




For:  0.05


0.8541


[[8422 1540]
 [1378 8660]]


             precision    recall  f1-score   support

          0       0.86      0.85      0.85      9962
          1       0.85      0.86      0.86     10038

avg / total       0.85      0.85      0.85     20000




For:  0.1


0.85445


[[8449 1513]
 [1398 8640]]


             precision    recall  f1-score   support

          0       0.86      0.85      0.85      9962
          1       0.85      0.86      0.86     10038

avg / total       0.85      0.85      0.85     20000




For:  0.25


0.8539


[[8438 1524]
 [1398 8640]]


             precision    recall  f1-score   support

          0       0.86      0.85      0.85      9962
          1       0.85      0.86   

### AT C= 0.1  ACCURACY SCORE IS 85.445

In [14]:
final_model = LogisticRegression(C=0.1)
final_model.fit(X_train, target)
predictions1 = final_model.predict(X_test)
print("Accuracy score is: ",accuracy_score(target,predictions1))
print("\n")
print("Confusion matrix is: ", confusion_matrix(target, predictions1))
print("\n")
print("Classification matrix is: ",classification_report(target, predictions1))

Accuracy score is:  0.87824


Confusion matrix is:  [[10945  1555]
 [ 1489 11011]]


Classification matrix is:               precision    recall  f1-score   support

          0       0.88      0.88      0.88     12500
          1       0.88      0.88      0.88     12500

avg / total       0.88      0.88      0.88     25000



### FINAL MODEL ACCURACY IS 87.78